In [1]:
import os
if not 'id_0123456789876543210' in locals():
    os.chdir(os.path.split(os.getcwd())[0])
    id_0123456789876543210 = None

In [2]:
from src.python.dataframe import Catalogue
from src.python.dataframe import filter
from os import path
import pandas as pd
import requests
import zipfile
import pickle
import shutil
import json
import os


import matplotlib.pyplot as plt
plt.style.use('ggplot')

def save_json(dict, file):
    with open(file, 'w', encoding='utf-8') as file:
        json.dump(dict, file, indent=4)


def load_json(file):
    with open(file, 'r') as file:
        return json.load(file)


def save_object(obj, file):
    with open(file, "wb") as file:
        pickle.dump(obj, file)


def load_object(file):
    with open(file, "rb") as file:
        loaded_object = pickle.load(file)
    return loaded_object


def download_file(url, file_name=None):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            if file_name is None:
                file_name = url.split('/')[-1]

            file_path = os.path.join(os.getcwd(), file_name)
            with open(file_path, 'wb') as file:
                file.write(response.content)

            print(f"File downloaded successfully as '{file_name}'")
            return file_path
        else:
            print(
                f"Error downloading the file. Status code: {response.status_code}")
            return None

    except Exception as e:
        print(f"Error during file download: {str(e)}")
        return None


def unzip_file(zip_file, destination_dir):

    if not os.path.exists(zip_file):
        print(f'Error: The file {zip_file} does not exist.')
        return

    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    try:
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(destination_dir)
        print(f'Files extracted to: {destination_dir}')
    except zipfile.BadZipFile:
        print(f'Error: The file {zip_file} is not a valid .zip file.')


def delete_file(file_path):
    try:
        os.remove(file_path)
        print(f'File deleted: {file_path}')
    except FileNotFoundError:
        print(f'Error: The file {file_path} does not exist.')
    except PermissionError:
        print(f'Error: Permission denied to delete the file {file_path}.')
    except OSError as e:
        print(f'Error deleting the file {file_path}: {e}')

def rename_file(file_path, new_name):
    try:
        directory = os.path.dirname(file_path)
        new_path = os.path.join(directory, new_name)
        os.rename(file_path, new_path)

        print(f'File renamed to: {new_path}')
    except FileNotFoundError:
        print(f'Error: The file {file_path} does not exist.')
    except OSError as e:
        print(f'Error renaming the file: {e}')

# Descarga de datos

In [3]:
urls = {
    2020: ['https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2020/COVID19MEXICO2020.zip'],
    2021: ['https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2021/COVID19MEXICO2021.zip'],
    2022: ['https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2022/COVID19MEXICO2022.zip'],
    2023: ['https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2023/COVID19MEXICO2023.zip'],
    2024: ['https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/01/datos_abiertos_covid19_02.01.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/01/datos_abiertos_covid19_09.01.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/01/datos_abiertos_covid19_16.01.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/01/datos_abiertos_covid19_23.01.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/01/datos_abiertos_covid19_30.01.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/02/datos_abiertos_covid19_06.02.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/02/datos_abiertos_covid19_13.02.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/02/datos_abiertos_covid19_20.02.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/02/datos_abiertos_covid19_27.02.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/03/datos_abiertos_covid19_05.03.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/03/datos_abiertos_covid19_12.03.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/03/datos_abiertos_covid19_19.03.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/03/datos_abiertos_covid19_26.03.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/04/datos_abiertos_covid19_02.04.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/04/datos_abiertos_covid19_09.04.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/04/datos_abiertos_covid19_16.04.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/04/datos_abiertos_covid19_23.04.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/04/datos_abiertos_covid19_30.04.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/05/datos_abiertos_covid19_07.05.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/05/datos_abiertos_covid19_14.05.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/05/datos_abiertos_covid19_21.05.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/05/datos_abiertos_covid19_28.05.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/06/datos_abiertos_covid19_04.06.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/06/datos_abiertos_covid19_11.06.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/06/datos_abiertos_covid19_18.06.2024.zip',
           'https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/06/datos_abiertos_covid19_25.06.2024.zip']}

for year, lst in urls.items():
    basis = path.join('data', 'covid', 'history', f'{year}')
    if path.exists(basis):
        shutil.rmtree(basis)
    os.makedirs(basis)
    for i, url in enumerate(lst):
        zip_name = path.split(url)[-1]
        zip_path = path.join(basis, zip_name)
        csv_name = f'{str(i).zfill(2)} ' + zip_name[:-3]+'csv'
        csv_path = path.join(basis, csv_name)
        if not download_file(url, zip_path) is None:
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                dwl_csv_path = path.join(basis, zip_ref.namelist()[0])
            unzip_file(zip_path, basis)
            delete_file(zip_path)
            rename_file(dwl_csv_path, csv_name)

File downloaded successfully as 'data\covid\history\2020\COVID19MEXICO2020.zip'
Files extracted to: data\covid\history\2020
File deleted: data\covid\history\2020\COVID19MEXICO2020.zip
File renamed to: data\covid\history\2020\00 COVID19MEXICO2020.csv
File downloaded successfully as 'data\covid\history\2021\COVID19MEXICO2021.zip'
Files extracted to: data\covid\history\2021
File deleted: data\covid\history\2021\COVID19MEXICO2021.zip
File renamed to: data\covid\history\2021\00 COVID19MEXICO2021.csv
File downloaded successfully as 'data\covid\history\2022\COVID19MEXICO2022.zip'
Files extracted to: data\covid\history\2022
File deleted: data\covid\history\2022\COVID19MEXICO2022.zip
File renamed to: data\covid\history\2022\00 COVID19MEXICO2022.csv
File downloaded successfully as 'data\covid\history\2023\COVID19MEXICO2023.zip'
Files extracted to: data\covid\history\2023
File deleted: data\covid\history\2023\COVID19MEXICO2023.zip
File renamed to: data\covid\history\2023\00 COVID19MEXICO2023.csv


# Limpieza de datos

Creación de catálogo para re-etiquetar los valores de las bases descargadas

In [4]:
entidad = load_json(
    path.join('data', 'covid', 'history', 'dictionary', 'entidad.json'))

catalogue = Catalogue()
SI_NO = {'1': 'SI', '2': 'NO', '97': 'NA'}
catalogue.add(column='EDAD', name='edad',
              function=lambda x: [int(x) for x in x])
catalogue.add(column='SEXO', name='sexo',
              category={'1': 'MUJER', '2': 'HOMBRE'})
catalogue.add(column='EMBARAZO', name='embarazo',
              category=SI_NO)
catalogue.add(column='NACIONALIDAD', name='nacionalidad',
              category={'1': 'MEXICANA', '2': 'EXTRANGERA'})
catalogue.add(column='MIGRANTE', name='migrante',
              category=SI_NO)
catalogue.add(column='INDIGENA', name='indigena',
              category=SI_NO)
catalogue.add(column='HABLA_LENGUA_INDIG', name='lengua_indigena',
              category=SI_NO)
catalogue.add(column='DIABETES', name='diabetes',
              category=SI_NO)
catalogue.add(column='EPOC', name='epoc',
              category=SI_NO)
catalogue.add(column='ASMA', name='asma',
              category=SI_NO)
catalogue.add(column='INMUSUPR', name='inmunosupresion',
              category=SI_NO)
catalogue.add(column='HIPERTENSION', name='hipertension',
              category=SI_NO)
catalogue.add(column='CARDIOVASCULAR', name='cardiovascular',
              category=SI_NO)
catalogue.add(column='OBESIDAD', name='obesidad',
              category=SI_NO)
catalogue.add(column='RENAL_CRONICA', name='renal_cronica',
              category=SI_NO)
catalogue.add(column='TABAQUISMO', name='tabaquismo',
              category=SI_NO)
catalogue.add(column='OTRA_COM', name='otra_comorbilidad',
              category=SI_NO)
# Variables regresoras
catalogue.add(column='TIPO_PACIENTE', name='tipo',
              category={'1': 'AMBULATORIO', '2': 'HOSPITALIZADO'})
catalogue.add(column='INTUBADO', name='intubado',
              category=SI_NO)
catalogue.add(column='NEUMONIA', name='neumonia',
              category=SI_NO)
catalogue.add(column='UCI', name='uci',
              category=SI_NO)
# Clasificación
catalogue.add(column='OTRO_CASO', name='contacto_covid',
              category=SI_NO)
catalogue.add(column='RESULTADO_LAB', name='laboratorio',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '3': 'PENDIENTE', '4': 'NO VALIDO', '97': 'NA'})
catalogue.add(column='RESULTADO_ANTIGENO', name='antigeno',
              category={'1': 'POSITIVO', '2': 'NEGATIVO', '97': 'NA'})
catalogue.add(column='CLASIFICACION_FINAL', name='clasificacion',
              category={'1': 'ASOCIACION', '2': 'DICTAMINACION', '3': 'CONFIRMACION',
                        '4': 'NO VALIDO', '5': 'PENDIENTE', '6': 'SOSPECHOSO', '7': 'NEGATIVO'})
# Fechas
catalogue.add(column='FECHA_INGRESO', name='fecha_ingreso',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_SINTOMAS', name='fecha_sintomas',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
catalogue.add(column='FECHA_DEF', name='fecha_defuncion',
              default=pd.NaT,
              exception='9999-99-99',
              function=lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
# Otros
catalogue.add(column='ORIGEN', name='origen',
              category={'1': 'USMER', '2': 'FUERA DE USMER'})
catalogue.add(column='SECTOR', name='sector',
              category={'1': 'CRUZ ROJA', '2': 'DIF', '3': 'ESTATAL', '4': 'IMSS', '5': 'IMSS-BIENESTAR',
                        '6': 'ISSSTE', '7': 'MUNICIPAL', '8': 'PEMEX', '9': 'PRIVADA', '10': 'SEDENA',
                        '11': 'SEMAR', '12': 'SSA', '13': 'UNIVERSITARIO'})
catalogue.add(column='ENTIDAD_UM', name='entidad',
              category=entidad)
catalogue.add(column='ENTIDAD_NAC', name='entidad_nacimiento',
              category=entidad)
catalogue.add(column='ENTIDAD_RES', name='entidad_residencia',
              category=entidad)
catalogue.add(column='MUNICIPIO_RES', name='municipio_residencia',
              function=lambda x: x)
catalogue.add(column='PAIS_NACIONALIDAD', name='pais_nacionalidad',
              function=lambda x: ['NE' if (x == 'SE DESCONOCE' or x == '99')
                                  else x.upper() for x in x])
catalogue.add(column='PAIS_ORIGEN', name='pais_origen',
              function=lambda x: ['NE' if x == 'SE DESCONOCE'
                                  else ('MÉXICO' if x == '97' else x.upper()) for x in x])

Lectura organización y re-etiquetación de bases históricas. La hubicación de los archivos debe de ser `data/covid/history`

In [5]:
# Juntar bases históricas
basis = path.join('data', 'covid', 'history')
superdata = None
for year in range(2024, 2019, -1):
    for file in reversed(os.listdir(path.join(basis, f'{year}'))):
        subdata = pd.read_csv(path.join(basis, f'{year}', file), dtype=str)
        if superdata is None:
            superdata = subdata
        else:
            superdata = pd.concat([superdata,
                                   subdata[~subdata.ID_REGISTRO.isin(superdata.ID_REGISTRO)]])

del subdata

# Guardar contadores iniciales
save_json({col: superdata[col].value_counts().to_dict()
           for col in superdata.columns if col != 'ID_REGISTRO'},
          path.join(basis, 'initial_counts.json'))


superdata = (filter(superdata, catalogue)
             .sort_values('fecha_ingreso')
             .reset_index(drop=True))

# Guardar contadores finales
save_json({col: {str(k): v for k, v in superdata[col].value_counts().to_dict().items()}
           for col in superdata.columns},
          path.join(basis, 'final_counts.json'))

del catalogue

# Clasificación de datos

Clasificación de bases en "positivos", "negativos" e "indeterminados"

In [6]:
for name, value in {'indeterminados': ['NO VALIDO', 'PENDIENTE', 'SOSPECHOSO'],
                    'positivos': ['ASOCIACION', 'DICTAMINACION', 'CONFIRMACION'],
                    'negativos': ['NEGATIVO']
                    }.items():
    locs = superdata.clasificacion.isin(value)
    basis = path.join('data', 'covid', 'cleanned')
    os.makedirs(basis, exist_ok=True)
    save_object(superdata[locs], path.join(basis, f'{name}.pkl'))
    superdata = superdata[~locs]
    print(f'{name[0].upper()}{name[1:]}: {sum(locs)}')

del superdata, locs, name, value

Indeterminados: 909078
Positivos: 7721082
Negativos: 11837091
